In [1]:
historical = "https://ieconnects.ie.edu/events?show=past"

import yaml
from selenium import webdriver
from ieconnects_login import login, DEFAULT_HYPER
from ieconnect_club import open_club_page
from communications.mail_helpers import open_mails_page, compose_email, select_email_members, fill_email_content, preview_and_send_mail
import time

with open("credentials.yaml") as f:
    credentials = yaml.load(f, Loader=yaml.FullLoader)
username = credentials["username"]
password = credentials["password"]

HYPER = DEFAULT_HYPER

# driver = webdriver.Firefox()


In [25]:
# make it so that it can be run from the command line with the
# driver not being visible
options = webdriver.FirefoxOptions()
if HYPER["headless"]:
    options.add_argument('--headless')
driver = webdriver.Firefox(options=options)


In [26]:
driver = login(driver, HYPER, username, password)
driver.implicitly_wait(3)
time.sleep(5)


LOGGING IN
STAGE 1 .. 2 .. 3 .. 4 .. 5
LOGGED IN


In [27]:
import pandas as pd
events = pd.DataFrame()

In [30]:
# get  from brw
from bs4 import BeautifulSoup
def get_events_raw(start=0, end=100):
    #api = "https://ieconnects.ie.edu/mobile_ws/v17/mobile_events_list?range=40&limit=40&filter1=past&filter4_contains=OR&filter4_notcontains=OR&order=undefined&search_word=&=&1708863225384="
    api = f"https://ieconnects.ie.edu/mobile_ws/v17/mobile_events_list?range={start}&limit={end}&filter1=past&filter4_contains=OR&filter4_notcontains=OR&order=undefined&search_word=&=&1708863225384="
    ap=driver.get(api)
    html = driver.page_source;
    # get body html w bs4
    soup = BeautifulSoup(html, "html.parser")
    # soup get body string
    body = soup.find("body")
    text = body.text
    import json
    data = json.loads(text);
    return data[1:]


import json
def many_events(count):
    # split into (0,n1), (n1,n2), (n2,n3), ... (nN, count)
    step = 100
    ranges = [(i, i+step) for i in range(0, count, step)]
    events = []
    for r in ranges:
        print(f"getting events {r[0]} to {r[1]}")
        try:
            events += get_events_raw(start=r[0], end=r[1])
            # cache every 100 events
            with open("cache.json", "w") as f:
                json.dump(events, f)
        except:
            print(f"failed to get events {r[0]} to {r[1]}")2


In [31]:
many = many_events(2000)

getting events 0 to 100
getting events 100 to 200
getting events 200 to 300
getting events 300 to 400
getting events 400 to 500
getting events 500 to 600
getting events 600 to 700
getting events 700 to 800
getting events 800 to 900
getting events 900 to 1000
getting events 1000 to 1100
getting events 1100 to 1200
getting events 1200 to 1300
getting events 1300 to 1400
getting events 1400 to 1500
getting events 1500 to 1600
getting events 1600 to 1700
getting events 1700 to 1800
getting events 1800 to 1900
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/usr/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_232503/3409730444.py", line 1, in <module>
    many = many_events(2000)
           ^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_232503/2035649841.py", line 26, in many_events
    events += get_events_raw(start=r[0], end=r[1])
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_232503/2035649841.py", line 6, in get_events_raw
    ap=driver.get(api)
       ^^^^^^^^^^^^^^^
  File "/home/velocitatem/.local/lib/python3.11/site-packages/selenium/webdriver/remote/webdriver.py", line 353, in get
    self.execute(Command.GET, {"url": url})
  File "/home/velocitatem/.local/lib/python3.11/site-packages/selenium/webdriver/remote/webdriver.py", line 344, in execute
    self.error_handler.check_response(response)
  File "/home/velocitatem/.local/lib/python3.11/site-packages

In [40]:
# load cache.json
import json
with open("cache.json", "r") as f:
    cache = json.load(f)
last100e = cache

{'fields': 'date_separator,eventId,eventUid,eventName,eventDates,eventCategory,eventLocation,clubId,clubLogin,clubName,eventAttendees,eventPicture,eventPriceRange,eventButtonLabel,eventBadges,totalTicketsSoldValue,checkbox_id,displayAttendees,eventUrl,displayType,registered,waiting_list,eventTags,coHostId,custom_time_instruction,checkin,registrationStatus,printTicket,eventTimezone,ariaEventDetails,ariaEventDetailsWithLocation,parentEventIds,all_results_hidden,hybrid,',
 'listingSeparator': None,
 'counter': '16815',
 'p0': 'false',
 'p1': '300367873',
 'p2': '9de91104ee5fb18e4f440bcd9495b02b',
 'p3': 'Weekly Running Practices',
 'p4': 'Sat, Feb 24, 20247 PM – 11:30 AM',
 'p5': 'Hybrid',
 'p6': 'Private Location (register to display)',
 'p7': '300002813',
 'p8': 'ierunners',
 'p9': 'IE Runners Club MIX',
 'p10': '0',
 'p11': '/upload/ie/2023/r2_image_upload_300315698_Final_Template__116124246.png',
 'p12': 'FREE',
 'p13': 'Register',
 'p14': '',
 'p15': '0.00',
 'p16': '300367873',
 'p1

In [33]:
len(last100e)

13169

In [57]:
def get_event_info(event_item):
    # some pn might be missing, if so, fill with None
    pks = [f"p{i}" for i in range(1, 34)]
    # rename keys
    i = 0
    event_info1 = {}
    fld = [a for a in event_item['fields'].split(",") if a != ""]
    for field in fld:
        print()
        event_info1[field] = event_item[f"p{i}"]
        i+=1
    event_item = {k: event_item.get(k, None) for k in pks}
    event_info = {
        "eventId": event_item["p1"],
        "eventUid": event_item["p2"],
        "eventName": event_item["p3"],
        "eventDates": event_item["p4"],
        "eventCategory": event_item["p5"],
        "eventLocation": event_item["p6"],
        "clubId": event_item["p7"],
        "clubLogin": event_item["p8"],
        "clubName": event_item["p9"],
        "eventAttendees": event_item["p10"],
        "eventPicture": event_item["p11"],
        "eventPriceRange": event_item["p12"],
        "eventButtonLabel": event_item["p13"],
        "eventBadges": event_item["p22"],
        "totalTicketsSoldValue": event_item["p15"],
        "checkbox_id": event_item["p16"],
        "eventUrl": event_item["p18"],
        "displayType": event_item["p19"],
        "registered": event_item["p20"],
        "waiting_list": event_item["p21"],
        "eventTags": event_item["p22"],
        "coHostId": event_item["p23"],
        "custom_time_instruction": event_item["p24"],
        "checkin": event_item["p25"],
        "registrationStatus": event_item["p26"],
        "printTicket": event_item["p27"],
        "eventTimezone": event_item["p28"],
        "ariaEventDetails": event_item["p29"],
        "ariaEventDetailsWithLocation": event_item["p30"],
        "parentEventIds": event_item["p31"],
        "all_results_hidden": event_item["p32"],
        "hybrid": event_item["p33"],
    }
    # compute event info and add to event_info1
    # diff:
    # extract ifo from eventDates dayofweek, month, day, year, time, timezone
    if event_info["eventDates"] is not None:
        try:
            event_info["eventDates"] = event_info["eventDates"].split(", ")
            dinfo = {
                "dayofweek": event_info["eventDates"][0],
                "month": event_info["eventDates"][1].split(" ")[0],
                "day": event_info["eventDates"][1].split(" ")[1],
                # first 4 digits are year
                "year": event_info["eventDates"][2][:4],
                "time": event_info["eventDates"][2][4:],
            }
            del event_info["eventDates"]
            event_info.update(dinfo)
        except:
            pass
    return event_info

get_event_info(last100e[1])



































set()


{'eventId': '300367873',
 'eventUid': '9de91104ee5fb18e4f440bcd9495b02b',
 'eventName': 'Weekly Running Practices',
 'eventCategory': 'Hybrid',
 'eventLocation': 'Private Location (register to display)',
 'clubId': '300002813',
 'clubLogin': 'ierunners',
 'clubName': 'IE Runners Club MIX',
 'eventAttendees': '0',
 'eventPicture': '/upload/ie/2023/r2_image_upload_300315698_Final_Template__116124246.png',
 'eventPriceRange': 'FREE',
 'eventButtonLabel': 'Register',
 'eventBadges': 'HybridAthletic CenterSportsWellness / wellbeing - Mental We...Wellness / wellbeing - Physical ...',
 'totalTicketsSoldValue': '0.00',
 'checkbox_id': '300367873',
 'eventUrl': '/rsvp?id=300367873',
 'displayType': 'event',
 'registered': '',
 'waiting_list': '',
 'eventTags': 'HybridAthletic CenterSportsWellness / wellbeing - Mental We...Wellness / wellbeing - Physical ...',
 'coHostId': '0',
 'custom_time_instruction': None,
 'checkin': '',
 'registrationStatus': '',
 'printTicket': '',
 'eventTimezone': '(GM

In [35]:
last100df = [get_event_info(e) for e in last100e]
last100df = pd.DataFrame(last100df)

In [37]:
last100df.describe()

,eventId,eventUid,eventName,eventCategory,eventLocation,clubId,clubLogin,clubName,eventAttendees,eventPicture,...,ariaEventDetailsWithLocation,parentEventIds,all_results_hidden,hybrid,dayofweek,month,day,year,time,eventDates
count,13169,13169,13169,12185,12185,12185,12185,12185,12185,12185,...,12185,12185,12185,12185,12160,12160,12160,12160,12160,25
unique,2679,2465,1424,5,690,172,172,172,236,1372,...,2443,55,1,2,7,10,31,2,371,1
top,,separator,False,On Campus,Online Event Zoom link,200000640,TalentCareers,Talent and Careers,0,/upload/ie/2022/r2_image_upload_200022780_0000...,...,,null,False,,Tue,Nov,14,2023,12 PM – 1 PM,[]
freq,25,984,984,4947,1739,3136,3136,3136,1714,364,...,25,9107,12185,11432,4752,3872,646,10302,1454,25


In [38]:
# save to csv
last100df.to_csv("13k.csv", index=False)